In [1]:
import HandTraking as ht
import cv2
import numpy as np
import time
import os

In [2]:
Folder_path = r"D:\Me\Ai\Computer vision projects\Virtua_drawing\versual_pain"

images = os.listdir(Folder_path)

print(images)

['1.jpg', '2.jpg', '3.jpg', '4.jpg']


In [3]:
heders = []

for image in images:
    img = cv2.imread(rf'{Folder_path}\{image}')
    heders.append(img)

print(len(heders))

4


In [4]:
header = heders[0]
DrawColor = (0,0,255)

brushThickness = 10
eraserThickness = 50

imgCanvas = np.zeros((720, 1280, 3), np.uint8)

xp, yp = 0,0


cap = cv2.VideoCapture(0)
cap.set(3,1240)
cap.set(4,720)
detector = ht.handDetector(trackCon = 0.9, detectionCon = 0.9)
while True:
    ret, img = cap.read()
    img = cv2.flip(img, 1)

    

    img = detector.findHands(img)
    lmList = detector.handposision(img,Draw = False)
    if len(lmList) != 0:
        #print(lmList)
        x1, y1 = lmList[8][1:]
        x2, y2 = lmList[12][1:]
    
        fingers = detector.fingersUp()
    
        if fingers[1] and fingers[2]:
            xp, yp = 0,0
            if y1 < 125:
                if 250 < x1 < 450:
                    header = heders[2]
                    DrawColor = (0, 0, 255)
                elif 550 < x1 < 750:
                    header = heders[1]
                    DrawColor = (255, 0, 0)
                elif 800 < x1 < 950:
                    header = heders[0]
                    DrawColor = (0, 255, 0)
                elif 1050 < x1 < 1200:
                    header = heders[3]
                    DrawColor = (0, 0, 0)
            cv2.rectangle(img, (x1, y1-25), (x2, y2+25), DrawColor, cv2.FILLED)
            
        if fingers[1] and fingers[2] == False:
            cv2.circle(img, (x1, y1), 15, DrawColor, cv2.FILLED)

            if xp == 0 and yp == 0:
                xp, yp = x1, y1
    
            if DrawColor == (0, 0, 0):
                cv2.line(img, (xp, yp), (x1, y1), DrawColor, eraserThickness)
                cv2.line(imgCanvas, (xp, yp), (x1, y1), DrawColor, eraserThickness)
            
            else:
                 cv2.line(img, (xp, yp), (x1, y1), DrawColor, brushThickness)
                 cv2.line(imgCanvas, (xp, yp), (x1, y1), DrawColor, brushThickness)
                
            xp, yp = x1, y1
            
        imgGray = cv2.cvtColor(imgCanvas, cv2.COLOR_BGR2GRAY)
        _, imgInv = cv2.threshold(imgGray, 50, 255, cv2.THRESH_BINARY_INV)
        imgInv = cv2.cvtColor(imgInv,cv2.COLOR_GRAY2BGR)
        img = cv2.bitwise_and(img,imgInv)
        img = cv2.bitwise_or(img,imgCanvas)

    
    img[0:120, 0:1240] = header
    cv2.imshow('img', img)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()